In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("../data/train.csv")

### EDA

In [45]:
#1. Check shape of dataset — Understand rows/columns count → Helps estimate dataset size.
def explore_shapes(df)->None:
    shapes = df.shape
    print(f"rows : {shapes[0]}")
    print(f"columns : {shapes[1]}")

In [48]:
#2. List all columns and data types — Identify numeric, categorical, date, and text features → Plan preprocessing.
def explore_types(df) -> dict:
    types = df.dtypes.value_counts().to_dict()
    for i,k in types.items():
        print(f"Total {k} of column type : {i}")
    type_to_column = {}
    for i,k in types.items():
        type_to_column[str(i)] = df.select_dtypes(include=[i]).columns.tolist()
    return type_to_column

In [140]:
# 3. Check null or missing values — Detect data quality issues → Plan imputation/removal.
def explore_null_or_missing(df:pd.DataFrame, type_to_column : dict)->tuple[pd.DataFrame, pd.DataFrame]:
    """
        Analyze and summarize missing (null) values in a dataset at both the *column level*
        ⚙️ **Parameters**
        ----------------------------------------------------------------------------
        df : pd.DataFrame  
            The input DataFrame to analyze. It should already be cleaned of any 
            structural issues (e.g., duplicated column names).

        type_to_column : dict  
            A mapping between logical data types and their corresponding columns.
            Typically derived from schema understanding or auto-detection.

            Example:
            ```
            {
                'numeric': ['age', 'salary', 'income'],
                'categorical': ['gender', 'city', 'profession'],
                'datetime': ['joining_date', 'exit_date']
            }
            ```
        ----------------------------------------------------------------------------
        📤 **Returns**
        ----------------------------------------------------------------------------
            tuple[pd.DataFrame, pd.DataFrame]
            Returns two summary DataFrames at column type level and column level:

    """
    numberOfRecord = len(df)
    columnLevelSummary = []
    typeLevelSummary = []
    for t, collist in type_to_column.items():
        numberOfcolumns = len(collist)
        missing = df[collist].isnull().sum()
        missing = missing[missing> 0]
        if len(missing) > 0:
            missing = missing.to_dict()
            typeLevelSummary.append({"column_type" : t, "column_with_missing_values" : len(missing), "total_columns" : numberOfcolumns, "percentage" : round(len(missing)/numberOfcolumns * 100, 2)})
            for col,v in missing.items():
                columnLevelSummary.append({"column_type" : t, "column" : col, "missing_record" : v, "total_record" : numberOfRecord, "percentage" : round(v/numberOfRecord * 100, 2)})


    typeLevelSummary = pd.DataFrame(typeLevelSummary)
    columnLevelSummary = pd.DataFrame(columnLevelSummary)
    return typeLevelSummary, columnLevelSummary

In [162]:
# 4. Check duplicate rows — Identify redundancy → Clean data.
def explore_duplicates(df : pd.DataFrame) -> None:
    """
    """
    isduplicate = df.duplicated().any()
    if isduplicate:
        print(f"Number of Duplicate records -> {df.duplicated().sum()}")
    else:
        print("No duplicate records Found")

In [163]:
explore_shapes(df)
type_to_column = explore_types(df)
typeLevelSummary, columnLevelSummary = explore_null_or_missing(df, type_to_column)
explore_duplicates(df)

rows : 1460
columns : 81
Total 43 of column type : object
Total 35 of column type : int64
Total 3 of column type : float64
No duplicate records Found


In [ ]:
# 5. Check unique values per column — Understand feature variety → Find categorical vs continuous.


In [161]:
df.duplicated().any

<bound method Series.any of 0       False
1       False
2       False
3       False
4       False
        ...  
1455    False
1456    False
1457    False
1458    False
1459    False
Length: 1460, dtype: bool>